<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on map
import folium
from folium.plugins import MarkerCluster

# SpaceX launch site data (latitude, longitude, name, and details)
launch_sites = [
    {
        "name": "Kennedy Space Center, Florida",
        "coordinates": [28.5729, -80.6489],
        "status": "Active",
        "details": "Primary launch site for Falcon 9 and Falcon Heavy. Hosts NASA missions."
    },
    {
        "name": "Cape Canaveral Space Force Station, Florida",
        "coordinates": [28.4889, -80.5778],
        "status": "Active",
        "details": "Used for ISS resupply missions and government payloads."
    },
    {
        "name": "Vandenberg Space Force Base, California",
        "coordinates": [34.6321, -120.6106],
        "status": "Active",
        "details": "West coast site for polar orbit launches."
    },
    {
        "name": "Boca Chica, Texas",
        "coordinates": [25.9972, -97.1550],
        "status": "Development",
        "details": "Starship development and test facility."
    }
]

# Create a base map centered on the US
spacex_map = folium.Map(location=[35, -100], zoom_start=4)

# Add marker cluster for better visualization of multiple markers
marker_cluster = MarkerCluster().add_to(spacex_map)

# Add each launch site to the map
for site in launch_sites:
    # Determine marker color based on status
    color = "green" if site["status"] == "Active" else "orange"
    
    # Create HTML content for the popup
    popup_html = f"""
    <b>{site['name']}</b><br>
    <hr>
    <b>Status:</b> {site['status']}<br>
    <b>Details:</b> {site['details']}<br>
    <b>Coordinates:</b> {site['coordinates'][0]:.4f}, {site['coordinates'][1]:.4f}
    """
    
    # Add marker to the map
    folium.Marker(
        location=site["coordinates"],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"Click for {site['name']} details",
        icon=folium.Icon(color=color, icon="rocket", prefix="fa")
    ).add_to(marker_cluster)

# Add a title to the map
title_html = '''
     <h3 align="center" style="font-size:16px"><b>SpaceX Launch Sites</b></h3>
     '''
spacex_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
spacex_map.save("spacex_launch_sites_map.html")
spacex_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

import folium
from folium.plugins import MarkerCluster, MiniMap
from folium.features import DivIcon

# SpaceX launch site data
launch_sites = [
    {
        "name": "Kennedy Space Center, Florida",
        "coordinates": [28.5729, -80.6489],
        "status": "Active",
        "details": "Primary launch site for Falcon 9 and Falcon Heavy. Hosts NASA missions.",
        "radius": 5000  # in meters
    },
    # ... (other launch sites remain the same)
]

# Create base map with proper attribution
spacex_map = folium.Map(
    location=[35, -100],
    zoom_start=4,
    tiles='Stamen Terrain',
    attr='Map data © <a href="https://www.openstreetmap.org/">OpenStreetMap</a> contributors, Tiles © <a href="https://stamen.com/">Stamen Design</a>'
)

# Rest of your code remains the same...
marker_cluster = MarkerCluster().add_to(spacex_map)

# Custom color function
def get_color(status):
    return '#3186cc' if status == "Active" else '#ff8c00'

# Add launch sites (same as before)
for site in launch_sites:
    folium.Circle(
        location=site["coordinates"],
        radius=site["radius"],
        color=get_color(site["status"]),
        fill=True,
        fill_color=get_color(site["status"]),
        fill_opacity=0.2,
        weight=2
    ).add_child(
        folium.Popup(f"<b>{site['name']}</b><br>Approximate launch area radius: {site['radius']}m")
    ).add_to(spacex_map)
    
    folium.map.Marker(
        site["coordinates"],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f"""<div style="font-size: 12pt; color:{get_color(site['status'])};">
                    <b>{site['name']}</b></div>"""
        )
    ).add_to(spacex_map)
    
    folium.Marker(
        location=site["coordinates"],
        popup=f"""
        <b>{site['name']}</b><hr>
        <b>Status:</b> {site['status']}<br>
        <b>Details:</b> {site['details']}<br>
        <b>Radius:</b> {site['radius']}m
        """,
        tooltip=f"Click for {site['name']} details",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Add controls and title
folium.LayerControl().add_to(spacex_map)
title_html = '''<h3 align="center" style="font-size:16px"><b>SpaceX Launch Sites with Operational Areas</b></h3>'''
spacex_map.get_root().html.add_child(folium.Element(title_html))

# Add minimap with attribution
minimap = MiniMap(tile_layer='Stamen Terrain', 
                 attr='Map data © OpenStreetMap contributors, Tiles © Stamen Design')
spacex_map.add_child(minimap)

# Display the map
spacex_map.save("spacex_launch_sites_enhanced.html")
spacex_map

<class 'ValueError'>: Custom tiles must have an attribution.

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [11]:
# Task 2: Mark the success/failed launches for each site on the map
import folium
from folium.plugins import MarkerCluster, MiniMap
from folium.features import DivIcon
import random  # For demo data - replace with your actual data

# Sample launch data (replace with your actual dataset)
launch_data = {
    "Kennedy Space Center, Florida": {
        "coordinates": [28.5729, -80.6489],
        "successful": 45,
        "failed": 3,
        "total": 48
    },
    "Cape Canaveral Space Force Station, Florida": {
        "coordinates": [28.4889, -80.5778],
        "successful": 32,
        "failed": 2,
        "total": 34
    },
    "Vandenberg Space Force Base, California": {
        "coordinates": [34.6321, -120.6106],
        "successful": 15,
        "failed": 1,
        "total": 16
    },
    "Boca Chica, Texas": {
        "coordinates": [25.9972, -97.1550],
        "successful": 4,
        "failed": 3,
        "total": 7
    }
}

# Create base map with proper attribution
launch_map = folium.Map(
    location=[35, -100],
    zoom_start=4,
    tiles='Stamen Terrain',
    attr='Map data © <a href="https://www.openstreetmap.org/">OpenStreetMap</a> contributors, Tiles © <a href="https://stamen.com/">Stamen Design</a>'
)

# Add marker cluster
marker_cluster = MarkerCluster().add_to(launch_map)

# Color thresholds for success rate
def get_success_color(success_rate):
    if success_rate >= 0.9: return '#2ecc71'  # Green
    elif success_rate >= 0.7: return '#f39c12'  # Orange
    else: return '#e74c3c'  # Red

# Add launch sites with success/failure markers
for site, data in launch_data.items():
    success_rate = data['successful'] / data['total']
    
    # Circle shows success rate (color) and launch volume (size)
    folium.Circle(
        location=data['coordinates'],
        radius=data['total'] * 200,  # Scale factor for visibility
        color=get_success_color(success_rate),
        fill=True,
        fill_color=get_success_color(success_rate),
        fill_opacity=0.6,
        weight=2,
        popup=f"""<b>{site}</b><br>
                 Success Rate: {success_rate:.1%}<br>
                 Successful: {data['successful']}<br>
                 Failed: {data['failed']}<br>
                 Total: {data['total']}"""
    ).add_to(launch_map)
    
    # Pie chart marker showing success/failure ratio
    success_angle = int(360 * success_rate)
    html = f"""
    <div style="font-size: 8pt; width: 60px; height: 60px;">
        <svg width="60" height="60">
            <circle cx="30" cy="30" r="25" fill="#2ecc71" 
                    stroke-width="2" stroke="#fff"
                    transform="rotate(-90, 30, 30)">
                <animate attributeName="stroke-dasharray"
                         values="0,{success_angle},0;{success_angle},0,0"
                         dur="1.5s" repeatCount="1" />
            </circle>
            <circle cx="30" cy="30" r="25" fill="none" 
                    stroke="#e74c3c" stroke-width="25" 
                    stroke-dasharray="{success_angle},360"
                    transform="rotate(-90, 30, 30)" />
            <text x="30" y="35" text-anchor="middle" 
                  font-weight="bold" fill="white">
                {data['total']}
            </text>
        </svg>
    </div>
    """
    
    folium.Marker(
        location=data['coordinates'],
        icon=DivIcon(
            icon_size=(60,60),
            icon_anchor=(30,30),
            html=html
        ),
        tooltip=f"{site}: {success_rate:.1%} success rate"
    ).add_to(marker_cluster)
    
    # Detailed marker with rocket icon
    folium.Marker(
        location=data['coordinates'],
        popup=f"""
        <b>{site}</b><hr>
        <b>Success Rate:</b> {success_rate:.1%}<br>
        <b>Successful Launches:</b> {data['successful']}<br>
        <b>Failed Launches:</b> {data['failed']}<br>
        <b>Total Launches:</b> {data['total']}
        """,
        icon=folium.Icon(color='black', icon='rocket', prefix='fa')
    ).add_to(launch_map)

# Add layer control and title
folium.LayerControl().add_to(launch_map)
title_html = '''<h3 align="center" style="font-size:16px"><b>SpaceX Launch Success Rates by Site</b></h3>'''
launch_map.get_root().html.add_child(folium.Element(title_html))

# Add minimap
minimap = MiniMap(tile_layer='Stamen Terrain',
                 attr='Map data © OpenStreetMap contributors, Tiles © Stamen Design')
launch_map.add_child(minimap)

# Add success rate legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 100px; 
                 border:2px solid grey; z-index:9999; font-size:12px;
                 background-color:white;
                 padding: 5px;">
         <b>Success Rate Legend</b><br>
         <i class="fa fa-circle" style="color:#2ecc71"></i> ≥90%<br>
         <i class="fa fa-circle" style="color:#f39c12"></i> 70-89%<br>
         <i class="fa fa-circle" style="color:#e74c3c"></i> <70%<br>
         <b>Circle Size:</b> Launch Volume
     </div>
     '''
launch_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map
launch_map.save("spacex_launch_success_map.html")
launch_map

<class 'ValueError'>: Custom tiles must have an attribution.

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [13]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [16]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Assuming spacex_df is your existing DataFrame
# Let's create a sample DataFrame if it doesn't exist (you can remove this part)
import pandas as pd
import numpy as np

# Assuming spacex_df is your existing DataFrame
# Let's create a sample DataFrame if it doesn't exist (you can remove this part)
spacex_df = pd.DataFrame({
    'Launch_Site': ['KSC', 'CCAFS', 'VAFB', 'Boca Chica'],
    'class': [1, 1, 0, 0],  # 1 = success, 0 = failure
    'lat': [28.5729, 28.4889, 34.6321, 25.9972],
    'lon': [-80.6489, -80.5778, -120.6106, -97.1550]
})

# Create marker_color column based on class values
spacex_df['marker_color'] = np.where(spacex_df['class'] == 1, 'green', 'red')

# Verify the result
print(spacex_df[['Launch_Site', 'class', 'marker_color']])

  Launch_Site  class marker_color
0         KSC      1        green
1       CCAFS      1        green
2        VAFB      0          red
3  Boca Chica      0          red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [17]:
# Assuming you already have site_map and marker_cluster created
# site_map = folium.Map(...)
# marker_cluster = MarkerCluster().add_to(site_map)

# For each row in the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Determine the icon properties based on launch success
    icon_properties = {
        'color': 'white',  # Outer circle color
        'icon_color': record['marker_color'],  # Inner icon color (green/red)
        'icon': 'rocket' if record['class'] == 1 else 'times',  # Different icons for success/failure
        'prefix': 'fa'  # Font Awesome icons
    }
    
    # Create the marker with customized icon
    marker = folium.Marker(
        location=[record['lat'], record['lon']],
        popup=f"""
        <b>Launch Site:</b> {record['Launch_Site']}<br>
        <b>Result:</b> {'Success' if record['class'] == 1 else 'Failure'}<br>
        <b>Date:</b> {record.get('Date', 'N/A')}<br>
        <b>Mission:</b> {record.get('Mission', 'N/A')}
        """,
        icon=folium.Icon(**icon_properties)
    )
    
    # Add the marker to the cluster
    marker_cluster.add_child(marker)

# Add the marker cluster to your site map
site_map.add_child(marker_cluster)

# Save or display the map
site_map.save('launch_results_map.html')
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [19]:
pip install geopy

<class 'OSError'>: Not available

In [18]:
# TASK 3: Calculate the distances between a launch site to its proximities
import pandas as pd
import numpy as np
from geopy.distance import geodesic  # More accurate than Haversine

# Sample launch sites data (replace with your actual data)
launch_sites = {
    "Kennedy Space Center": {"lat": 28.5729, "lon": -80.6489},
    "Cape Canaveral SFS": {"lat": 28.4889, "lon": -80.5778},
    "Vandenberg SFB": {"lat": 34.6321, "lon": -120.6106},
    "Boca Chica": {"lat": 25.9972, "lon": -97.1550}
}

# Nearby cities/landmarks for each launch site
proximities = {
    "Kennedy Space Center": [
        {"name": "Orlando", "lat": 28.5383, "lon": -81.3792},
        {"name": "Cocoa Beach", "lat": 28.3200, "lon": -80.6076}
    ],
    "Cape Canaveral SFS": [
        {"name": "Titusville", "lat": 28.6122, "lon": -80.8076},
        {"name": "Port Canaveral", "lat": 28.4106, "lon": -80.6189}
    ],
    "Vandenberg SFB": [
        {"name": "Lompoc", "lat": 34.6391, "lon": -120.4579},
        {"name": "Santa Maria", "lat": 34.9530, "lon": -120.4357}
    ],
    "Boca Chica": [
        {"name": "Brownsville", "lat": 25.9018, "lon": -97.4975},
        {"name": "South Padre Island", "lat": 26.1118, "lon": -97.1681}
    ]
}

# Calculate distances and create a DataFrame
distance_data = []

for site, coords in launch_sites.items():
    for proximity in proximities[site]:
        # Calculate distance using geodesic (most accurate)
        site_coord = (coords["lat"], coords["lon"])
        prox_coord = (proximity["lat"], proximity["lon"])
        distance_km = geodesic(site_coord, prox_coord).kilometers
        distance_mi = geodesic(site_coord, prox_coord).miles
        
        distance_data.append({
            "Launch Site": site,
            "Proximity": proximity["name"],
            "Distance (km)": round(distance_km, 2),
            "Distance (miles)": round(distance_mi, 2)
        })

# Create DataFrame
distances_df = pd.DataFrame(distance_data)

# Display the results
print(distances_df)

# Optional: Add to your Folium map
def add_distance_lines(site_map):
    for site, coords in launch_sites.items():
        for proximity in proximities[site]:
            folium.PolyLine(
                locations=[(coords["lat"], coords["lon"]), 
                         (proximity["lat"], proximity["lon"])],
                color='blue',
                weight=1,
                popup=f"{site} to {proximity['name']}: {geodesic((coords['lat'], coords['lon']), (proximity['lat'], proximity['lon'])).km:.1f} km"
            ).add_to(site_map)
    return site_map

# Add to your existing map
site_map = add_distance_lines(site_map)
site_map.save('launch_sites_with_distances.html')

<class 'ModuleNotFoundError'>: No module named 'geopy'

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [20]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [21]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [22]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

import folium
from folium.plugins import MousePosition
from folium.features import DivIcon
import math

# Launch site data
launch_sites = {
    "Kennedy Space Center": {"lat": 28.5729, "lon": -80.6489},
    "Cape Canaveral SFS": {"lat": 28.4889, "lon": -80.5778},
    "Vandenberg SFB": {"lat": 34.6321, "lon": -120.6106},
    "Boca Chica": {"lat": 25.9972, "lon": -97.1550}
}

# Known closest coastline points (you can update these with more precise coordinates)
coastline_points = {
    "Kennedy Space Center": {"lat": 28.56367, "lon": -80.57163},
    "Cape Canaveral SFS": {"lat": 28.4889, "lon": -80.5778},  # Already on coastline
    "Vandenberg SFB": {"lat": 34.6321, "lon": -120.6106},     # On coastline
    "Boca Chica": {"lat": 25.9972, "lon": -97.1550}           # On coastline
}

# Haversine distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    return R * c

# Create map centered on US launch sites
site_map = folium.Map(location=[30, -100], zoom_start=4)

# Add MousePosition to get coordinates
MousePosition(
    position='topright',
    separator=' | ',
    prefix="Coordinates:",
    empty_string="Mouse over map for coordinates",
    num_digits=5
).add_to(site_map)

# Add launch sites and coastline markers
for site, coords in launch_sites.items():
    coastline = coastline_points[site]
    distance = calculate_distance(
        coords["lat"], coords["lon"],
        coastline["lat"], coastline["lon"]
    )
    
    # Add launch site marker
    folium.Marker(
        location=[coords["lat"], coords["lon"]],
        popup=f"{site}<br>Distance to coast: {distance:.2f} km",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(site_map)
    
    # Add coastline marker (only if not the same as launch site)
    if (coords["lat"] != coastline["lat"]) or (coords["lon"] != coastline["lon"]):
        folium.Marker(
            location=[coastline["lat"], coastline["lon"]],
            popup=f"Closest coastline to {site}<br>{distance:.2f} km away",
            icon=folium.Icon(color='blue', icon='water', prefix='fa')
        ).add_to(site_map)
        
        # Add line connecting launch site to coastline
        folium.PolyLine(
            locations=[(coords["lat"], coords["lon"]), 
                     (coastline["lat"], coastline["lon"])],
            color='green',
            weight=2,
            dash_array='5,5',
            popup=f"Distance: {distance:.2f} km"
        ).add_to(site_map)

# Add distance calculation control
distance_control = """
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 200px; height: 100px; 
            border:2px solid grey; z-index:9999; font-size:12px;
            background-color:white; padding: 10px;">
    <b>Distance Calculation</b><br>
    1. Hover mouse to find coastline<br>
    2. Right-click to mark point<br>
    3. Distance will show in popup
</div>
"""
site_map.get_root().html.add_child(folium.Element(distance_control))

# Save and display
site_map.save('launch_sites_coastline.html')
site_map

In [23]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium.features import DivIcon
import math

# Launch site data with closest coastline points
launch_data = {
    "Kennedy Space Center": {
        "site_coords": [28.5729, -80.6489],
        "coast_coords": [28.56367, -80.57163]
    },
    "Cape Canaveral SFS": {
        "site_coords": [28.4889, -80.5778],
        "coast_coords": [28.4889, -80.5778]  # Already on coastline
    },
    "Vandenberg SFB": {
        "site_coords": [34.6321, -120.6106],
        "coast_coords": [34.6321, -120.6106]  # On coastline
    },
    "Boca Chica": {
        "site_coords": [25.9972, -97.1550],
        "coast_coords": [25.9972, -97.1550]  # On coastline
    }
}

# Haversine distance calculation
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    return R * 2 * math.asin(math.sqrt(a))

# Create map
site_map = folium.Map(location=[30, -100], zoom_start=4)

# Process each launch site
for site_name, data in launch_data.items():
    site_lat, site_lon = data["site_coords"]
    coast_lat, coast_lon = data["coast_coords"]
    distance = calculate_distance(site_lat, site_lon, coast_lat, coast_lon)
    
    # Add launch site marker
    folium.Marker(
        location=data["site_coords"],
        popup=f"<b>{site_name}</b><br>Coordinates: {site_lat:.5f}, {site_lon:.5f}",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(site_map)
    
    # Only add coastline marker if different from launch site
    if distance > 0.1:  # Only if more than 100m apart
        # Add coastline marker with distance label
        distance_marker = folium.Marker(
            location=data["coast_coords"],
            icon=DivIcon(
                icon_size=(150,30),
                icon_anchor=(75,15),
                html=f"""<div style="font-size: 12pt; color:#d35400; background:white; 
                         border:1px solid black; border-radius:3px; padding:2px;">
                         <b>{distance:.2f} KM</b></div>"""
            )
        ).add_to(site_map)
        
        # Add connecting line
        folium.PolyLine(
            locations=[data["site_coords"], data["coast_coords"]],
            color='blue',
            weight=2,
            dash_array='5,5',
            popup=f"{site_name} to Coastline: {distance:.2f} km"
        ).add_to(site_map)

# Add map controls
folium.LayerControl().add_to(site_map)

# Add title
title_html = '''
     <h3 align="center" style="font-size:16px"><b>SpaceX Launch Sites - Distance to Coastline</b></h3>
     '''
site_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
site_map.save('launch_sites_coastline_distances.html')
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [24]:

import folium
from folium.features import DivIcon
import math

# Launch site data with coordinates and closest coastline points
launch_data = {
    "Kennedy Space Center": {
        "site_coords": [28.5729, -80.6489],
        "coast_coords": [28.56367, -80.57163],
        "color": "red"
    },
    "Cape Canaveral SFS": {
        "site_coords": [28.4889, -80.5778],
        "coast_coords": [28.4889, -80.5778],  # Already on coastline
        "color": "blue"
    },
    "Vandenberg SFB": {
        "site_coords": [34.6321, -120.6106],
        "coast_coords": [34.6321, -120.6106],  # On coastline
        "color": "green"
    },
    "Boca Chica": {
        "site_coords": [25.9972, -97.1550],
        "coast_coords": [25.9972, -97.1550],  # On coastline
        "color": "orange"
    }
}

# Create base map
site_map = folium.Map(location=[30, -100], zoom_start=4)

# Process each launch site
for site_name, data in launch_data.items():
    site_lat, site_lon = data["site_coords"]
    coast_lat, coast_lon = data["coast_coords"]
    distance = math.sqrt((site_lat - coast_lat)**2 + (site_lon - coast_lon)**2) * 111  # Approx km
    
    # Add launch site marker
    folium.Marker(
        location=data["site_coords"],
        popup=f"<b>{site_name}</b><br>Coordinates: {site_lat:.5f}, {site_lon:.5f}",
        icon=folium.Icon(color=data["color"], icon='rocket', prefix='fa')
    ).add_to(site_map)
    
    # Only add coastline marker and line if not the same point
    if distance > 0.1:  # More than 100m apart
        # Add coastline marker
        folium.Marker(
            location=data["coast_coords"],
            icon=folium.Icon(color='lightblue', icon='water', prefix='fa')
        ).add_to(site_map)
        
        # Create PolyLine between launch site and coastline
        lines = folium.PolyLine(
            locations=[data["site_coords"], data["coast_coords"]],
            color=data["color"],
            weight=2,
            dash_array='5, 5',
            opacity=0.8,
            popup=f"Distance: {distance:.2f} km"
        )
        site_map.add_child(lines)
        
        # Add distance label at midpoint
        midpoint = [(site_lat + coast_lat)/2, (site_lon + coast_lon)/2]
        folium.map.Marker(
            midpoint,
            icon=DivIcon(
                icon_size=(150,30),
                icon_anchor=(75,15),
                html=f"""<div style="font-size:10pt; color:{data['color']}; 
                         background:white; border:1px solid {data['color']}; 
                         border-radius:3px; padding:2px; text-align:center;">
                         <b>{distance:.2f} km</b></div>"""
            )
        ).add_to(site_map)

# Add map controls and title
folium.LayerControl().add_to(site_map)
title_html = '''<h3 align="center" style="font-size:16px"><b>SpaceX Launch Sites - Distance to Coastline</b></h3>'''
site_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
site_map.save('launch_sites_coastline_lines.html')
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [25]:
import folium
from folium.features import DivIcon
import math

# Launch site data with infrastructure points
launch_data = {
    "Kennedy Space Center": {
        "coordinates": [28.5729, -80.6489],
        "infrastructure": [
            {"type": "city", "name": "Titusville", "coords": [28.6122, -80.8076], "color": "blue"},
            {"type": "highway", "name": "I-95", "coords": [28.5275, -80.8512], "color": "orange"},
            {"type": "railway", "name": "Florida East Coast Railway", "coords": [28.5233, -80.7991], "color": "green"}
        ]
    },
    "Vandenberg SFB": {
        "coordinates": [34.6321, -120.6106],
        "infrastructure": [
            {"type": "city", "name": "Lompoc", "coords": [34.6391, -120.4579], "color": "blue"},
            {"type": "highway", "name": "US-101", "coords": [34.7210, -120.4890], "color": "orange"},
            {"type": "railway", "name": "Coast Line", "coords": [34.7038, -120.5260], "color": "green"}
        ]
    }
}

def calculate_distance(coord1, coord2):
    """Calculate distance between two coordinates in kilometers"""
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    return math.sqrt((lat2-lat1)**2 + (lon2-lon1)**2) * 111.32  # Approximate km

# Create base map
site_map = folium.Map(location=[32, -110], zoom_start=5)

# Custom icons
icon_dict = {
    "city": "building",
    "highway": "road",
    "railway": "train"
}

# Process each launch site
for site_name, data in launch_data.items():
    # Add launch site marker
    folium.Marker(
        location=data["coordinates"],
        popup=f"<b>{site_name}</b>",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(site_map)
    
    # Add infrastructure markers and lines
    for infra in data["infrastructure"]:
        distance = calculate_distance(data["coordinates"], infra["coords"])
        
        # Add infrastructure marker
        folium.Marker(
            location=infra["coords"],
            popup=f"<b>{infra['name']}</b><br>Type: {infra['type']}<br>Distance: {distance:.1f} km",
            icon=folium.Icon(color=infra["color"], icon=icon_dict[infra["type"]], prefix='fa')
        ).add_to(site_map)
        
        # Create PolyLine between launch site and infrastructure
        folium.PolyLine(
            locations=[data["coordinates"], infra["coords"]],
            color=infra["color"],
            weight=2,
            opacity=0.7,
            popup=f"Distance to {infra['name']}: {distance:.1f} km"
        ).add_to(site_map)
        
        # Add distance label at midpoint
        midpoint = [
            (data["coordinates"][0] + infra["coords"][0])/2,
            (data["coordinates"][1] + infra["coords"][1])/2
        ]
        folium.map.Marker(
            midpoint,
            icon=DivIcon(
                icon_size=(120,30),
                icon_anchor=(60,15),
                html=f"""<div style="font-size:10pt; color:{infra['color']}; 
                         background:white; border:1px solid {infra['color']}; 
                         border-radius:3px; padding:2px; text-align:center;">
                         <b>{distance:.1f} km</b></div>"""
            )
        ).add_to(site_map)

# Add map controls and title
folium.LayerControl().add_to(site_map)
title_html = '''<h3 align="center" style="font-size:16px"><b>SpaceX Launch Sites - Distance to Infrastructure</b></h3>'''
site_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
site_map.save('launch_sites_infrastructure.html')
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
